In [0]:
# Imports needed
import os, re, torch, html, tempfile, copy, json, math, shutil, tarfile, tempfile, sys, random, pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

Upload the datasets to Google Drive. 
This allows access to your Google Drive from this notebook.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

# Class Definitions:
## Data Preprocessing

In [0]:
sfu_data = '/content/gdrive/My Drive/multilingual_BERT_negations/data/SFU_Review_Corpus_Negation_Speculation'
frac_no_cue_sents = 1.0
TASK = 'negation'

In [0]:
def sfu_review(f_path, cue_sents_only=False, frac_no_cue_sents = 1.0):
    # print(f_path)
    file = open(f_path, encoding = 'utf-8')
    sentences = []
    for s in file:
        print(s)
        sentences+=re.split("(<.*?>)", html.unescape(s))
    cue_sentence = []
    cue_cues = []
    scope_cues = []
    scope_scopes = []
    scope_sentence = []
    sentence = []
    cue = {}
    scope = {}
    in_scope = []
    in_cue = []
    word_num = 0
    c_idx = []
    cue_only_data = []
    s_idx = []
    in_word = 0
    print(len(sentences))
    print(sentences)
    for token in sentences:
        if token == '':
            continue
        elif token == '<W>':
            in_word = 1
        elif token == '</W>':
            in_word = 0
            word_num += 1
        elif '<cue' in token:
            if TASK in token:
                in_cue.append(int(re.split('(ID=".*?")',token)[1][4:-1]))
                c_idx.append(int(re.split('(ID=".*?")',token)[1][4:-1]))
                cue[c_idx[-1]] = []
        elif '</cue' in token:
            in_cue = in_cue[:-1]
        elif '<xcope' in token:
            continue
        elif '</xcope' in token:
            in_scope = in_scope[:-1]
        elif '<ref' in token:
            in_scope.append([int(i) for i in re.split('(SRC=".*?")',token)[1][5:-1].split(' ')])
            s_idx.append([int(i) for i in re.split('(SRC=".*?")',token)[1][5:-1].split(' ')])
            for i in s_idx[-1]:
                scope[i] = []
        elif '</SENTENCE' in token:
            if len(cue.keys())==0:
                cue_only_data.append([sentence, [3]*len(sentence)])
            else:
                cue_sentence.append(sentence)
                cue_cues.append([3]*len(sentence))
                for i in cue.keys():
                    scope_sentence.append(sentence)
                    scope_cues.append([3]*len(sentence))
                    if len(cue[i])==1:
                        cue_cues[-1][cue[i][0]] = 1
                        scope_cues[-1][cue[i][0]] = 1
                    else:
                        for c in cue[i]:
                            cue_cues[-1][c] = 2
                            scope_cues[-1][c] = 2
                    scope_scopes.append([0]*len(sentence))
                    if i in scope.keys():
                        for s in scope[i]:
                            scope_scopes[-1][s] = 1
            sentence = []
            cue = {}
            scope = {}
            in_scope = []
            in_cue = []
            word_num = 0
            in_word = 0
            c_idx = []
            s_idx = []
        elif '<' not in token:
            if in_word == 1:
                if len(in_cue)!=0:
                    for i in in_cue:
                        cue[i].append(word_num)
                if len(in_scope)!=0:
                    for i in in_scope:
                        for j in i:
                            scope[j].append(word_num)
                sentence.append(token)
    cue_only_samples = random.sample(cue_only_data, k=int(frac_no_cue_sents*len(cue_only_data)))
    cue_only_sents = [i[0] for i in cue_only_samples]
    cue_only_cues = [i[1] for i in cue_only_samples]
    return [(cue_sentence+cue_only_sents, cue_cues+cue_only_cues),(scope_sentence, scope_cues, scope_scopes)]
      

In [0]:
sfu_cues = [[], []]
sfu_scopes = [[], [], []]
for dir_name in os.listdir(sfu_data):
    if '.' not in dir_name:
        for f_name in os.listdir(sfu_data+"//"+dir_name):
            r_val = sfu_review(sfu_data+"//"+dir_name+'//'+f_name, frac_no_cue_sents=frac_no_cue_sents)
            sfu_cues = [a+b for a,b in zip(sfu_cues, r_val[0])]
            sfu_scopes = [a+b for a,b in zip(sfu_scopes, r_val[1])]

scope_sents, scope_cues, data_scope = sfu_scopes

In [0]:
for item in zip(scope_sents, scope_cues, data_scope):
  print(item[0])
  print(item[1])
  print(item[2])
  print()

In [0]:
method = SCOPE_METHOD
do_lower_case = True
tokenizer = BertTokenizer.from_pretrained(SCOPE_MODEL, do_lower_case=do_lower_case, cache_dir='bert_tokenizer')

In [0]:
def preprocess_data(scope_sents, scope_cues, data_scope):
  dl_sents = scope_sents
  dl_cues = scope_cues
  dl_scopes = data_scope
    
  sentences = [" ".join([s for s in sent]) for sent in dl_sents]
  mytexts = []
  mylabels = []
  mycues = []
  mymasks = []
  if do_lower_case == True:
      sentences_clean = [sent.lower() for sent in sentences]
  else:
      sentences_clean = sentences
  
  for sent, tags, cues in zip(sentences_clean,dl_scopes, dl_cues):
      new_tags = []
      new_text = []
      new_cues = []
      new_masks = []
      for word, tag, cue in zip(sent.split(),tags,cues):
          sub_words = tokenizer._tokenize(word)
          for count, sub_word in enumerate(sub_words):
              mask = 1
              if count > 0:
                  mask = 0
              new_masks.append(mask)
              new_tags.append(tag)
              new_cues.append(cue)
              new_text.append(sub_word)
      mymasks.append(new_masks)
      mytexts.append(new_text)
      mylabels.append(new_tags)
      mycues.append(new_cues)

  final_sentences = []
  final_labels = []
  final_masks = []

  for sent,cues,labels,masks in zip(mytexts, mycues, mylabels, mymasks):
    temp_sent = []
    temp_label = []
    temp_masks = []
    first_part = 0
    for token,cue,label,mask in zip(sent,cues,labels,masks):
        if cue!=3:
            if first_part == 0:
                first_part = 1
                temp_sent.append(f'[unused{cue+1}]')
                temp_masks.append(1)
                temp_label.append(0)
                temp_sent.append(token)
                temp_masks.append(0)
                temp_label.append(label)
                continue
            temp_sent.append(f'[unused{cue+1}]')
            temp_masks.append(0)
            temp_label.append(0)
        else:
            first_part = 0
        temp_masks.append(mask)
        temp_sent.append(token)
        temp_label.append(label)
    temp_sent.insert(0, '[CLS]')
    temp_sent.append('[SEP]')
    temp_masks.append(0)
    temp_label.append(0)
    temp_label.insert(0, 0)
    temp_masks.insert(0, 0)
    final_sentences.append(temp_sent)
    final_labels.append(temp_label)
    final_masks.append(temp_masks)


  return final_sentences, final_labels, final_masks         

In [0]:
final_sentences, final_labels, final_masks = preprocess_data(scope_sents, scope_cues, data_scope)

In [0]:
input_ids = pad_sequences([[tokenizer._convert_token_to_id(word) for word in txt] for txt in final_sentences],
                                      maxlen=MAX_LEN, dtype="long", truncating="post", padding="post").tolist()

In [0]:
tags = pad_sequences(final_labels, maxlen=MAX_LEN, value=0, padding="post", dtype="long", truncating="post").tolist()

In [0]:
finalest_masks = pad_sequences(final_masks, maxlen=MAX_LEN, value=0, padding='post', dtype='long', truncating='post').tolist()

In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [20]:
print(scope_sents[3])
print(scope_cues[3])
print('True scope labels:', data_scope[3])

['There\x92s', 'also', 'plenty', 'of', 'historical', 'information', ',', 'which', 'is', 'integrated', 'smoothly', 'into', 'the', 'prose', ';', 'remarkably', ',', 'even', 'when', 'inserted', 'into', 'dialogue', ',', 'the', 'explanatory', 'asides', 'don\x92t', 'interrupt', 'the', 'flow', '.']
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3]
True scope labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]


In [0]:
outfile = format('/content/gdrive/My Drive/multilingual_BERT_negations/data/sfu_sentences.txt')

In [40]:
count_sents = 0
hash_values = set()
with open(outfile, 'w', encoding='utf8') as outf:
  for sent in scope_sents:
    sent = ' '.join(sent)
    sent_hash = hash(sent)
    count_sents += 1
    if sent_hash not in hash_values:
      hash_values.add(sent_hash)
      if all(token in sent.split() for token in ["n't", 'no']):
        print(sent)
    # outf.write(' '.join(sent))

count_sents

There is no flowery dialog , and time is n't wasted trying to establish a hot relationship between the two lead characters .
When you first open the book , reading the Prologue , it grips you , right then , right there , so early on , you have no choice but to read , quickly , with anticipation , knowing in your heart that Alex has to be triumphant ! , does n't he ?
Additionally , the G5 no longer supports the Apple Pro Speaker connector , so I ca n't use the Griffin ProSpeaker Cable to connect speakers to the machine .
For now , I 've banned my kids from even coming within 5 feet of this baby , though I know I wo n't be able to enforce the " no kid zone " for long .
A long time ago I returned a new desktop PC to Gateway because I could n't operate it intuitively ( my fault ) , and Gateway graciously took it back , no questions asked .
My iMac was trouble from the start and I got no help from Apple other than advice , and after 90 days I could n't even get that unless I forked over an 

3528

## INPUT for BERT

In [0]:
x = 3
print(final_sentences[x])
print(len(final_sentences[x]))
print()
print("input_ids: ", input_ids[x])
print("scope ids: ", tags[x])
print("true token masks: ", finalest_masks[x])
print('attention masks: ', attention_masks[x])

['[CLS]', 'but', 'the', 'morning', 'paper', 'was', '[unused1]', 'un', '[unused1]', '##int', '[unused1]', '##eres', '[unused1]', '##ting', '.', '[SEP]']
16

input_ids:  [101, 2021, 1996, 2851, 3259, 2001, 2, 4895, 2, 18447, 2, 18702, 2, 3436, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
scope ids:  [0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
true token mas